### LEFT and RIGHT JOINs

In [1]:
import pandas as pd
import sqlite3

In [2]:
def sql_query(query):
    c = cur.execute(query)
    df = pd.DataFrame(c.fetchall(), 
                      columns=[description[0] for description in c.description])
    return df

In [3]:
conn = sqlite3.connect('leaders.sqlite')
cur = conn.cursor()

In [4]:
sql_query('''SELECT p1.country, prime_minister, president
FROM prime_ministers as p1
LEFT JOIN presidents as p2
ON p1.country = p2.country''')

,country,prime_minister,president
0,Egypt,Sherif Ismail,Abdel Fattah el-Sisi
1,Portugal,Antonio Costa,Marcelo Rebelo de Sousa
2,Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
3,Haiti,Jack Guy Lafontant,Jovenel Moise
4,India,Narendra Modi,None
5,Australia,Malcolm Turnbull,None
6,Norway,Erna Solberg,None
7,Brunei,Hassanal Bolkiah,None
8,Oman,Qaboos bin Said al Said,None
9,Spain,Mariano Rajoy,None


In [5]:
conn.close()

In [6]:
conn = sqlite3.connect('countries.sqlite')
cur = conn.cursor()

In [7]:
sql_query('''-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.country_name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1		
  -- Join to right table (with alias)
  LEFT JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC;''').head(3)

,city,code,country,region,city_proper_pop
0,Harare,ZWE,Zimbabwe,Eastern Africa,1606000
1,Lusaka,ZMB,Zambia,Eastern Africa,1742979
2,Cape Town,ZAF,South Africa,Southern Africa,3740026


In [8]:
sql_query('''/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.country_name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on fields
    ON c.code = l.code
-- 4. Order by descending country
ORDER BY country DESC;''').head(3)

,country,local_name,language,percent
0,Zimbabwe,Zimbabwe,Chewa,None
1,Zimbabwe,Zimbabwe,Chibarwe,None
2,Zimbabwe,Zimbabwe,English,None


In [9]:
sql_query('''-- Select fields
SELECT region, AVG(gdp_percapita) AS avg_gdp
-- From countries (alias as c)
FROM countries AS c
  -- Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- Match on code fields
    ON c.code = e.code
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region
-- Order by descending avg_gdp
ORDER BY avg_gdp DESC;''').head(3)

,region,avg_gdp
0,Western Europe,58130.962857
1,Nordic Countries,57073.998000
2,North America,47911.510000


In [10]:
# sql_query('''
# SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
#        indep_year, languages.name AS language, percent
# FROM languages
#   RIGHT JOIN countries
#     ON languages.code = countries.code
#   RIGHT JOIN cities
#     ON countries.code = cities.country_code
# ORDER BY city, language;''')

In [11]:
conn.close()

### FULL JOINS

In [12]:
conn = sqlite3.connect('leaders.sqlite')
cur = conn.cursor()

In [13]:
sql_query('''SELECT p1.country AS pm_co, p2.country AS pres_co, prime_minister, president
FROM presidents AS p2
LEFT OUTER JOIN prime_ministers AS p1
ON p1.country = p2.country
UNION
SELECT p1.country AS pm_co, p2.country AS pres_co, prime_minister, president
FROM prime_ministers AS p1
LEFT OUTER JOIN presidents AS p2 
ON p1.country = p2.country
ORDER BY pm_co DESC;''')

,pm_co,pres_co,prime_minister,president
0,Vietnam,Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
1,Spain,None,Mariano Rajoy,None
2,Portugal,Portugal,Antonio Costa,Marcelo Rebelo de Sousa
3,Oman,None,Qaboos bin Said al Said,None
4,Norway,None,Erna Solberg,None
5,India,None,Narendra Modi,None
6,Haiti,Haiti,Jack Guy Lafontant,Jovenel Moise
7,Egypt,Egypt,Sherif Ismail,Abdel Fattah el-Sisi
8,Brunei,None,Hassanal Bolkiah,None
9,Australia,None,Malcolm Turnbull,None


In [14]:
sql_query('''SELECT prime_minister, president
FROM prime_ministers as p1
CROSS JOIN presidents as p2
WHERE p1.continent IN ('North America', 'Oceania');''')

,prime_minister,president
0,Jack Guy Lafontant,Abdel Fattah el-Sisi
1,Jack Guy Lafontant,Marcelo Rebelo de Sousa
2,Jack Guy Lafontant,Jovenel Moise
3,Jack Guy Lafontant,Jose Mujica
4,Jack Guy Lafontant,Ellen Johnson Sirleaf
5,Jack Guy Lafontant,Michelle Bachelet
6,Jack Guy Lafontant,Tran Dai Quang
7,Malcolm Turnbull,Abdel Fattah el-Sisi
8,Malcolm Turnbull,Marcelo Rebelo de Sousa
9,Malcolm Turnbull,Jovenel Moise


In [16]:
conn.close()

In [17]:
conn = sqlite3.connect('countries.sqlite')
cur = conn.cursor()

In [18]:
sql_query('''-- 4. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c        
  -- 2. Join to languages (alias as l)
  INNER JOIN languages AS l
  ON c.country_code = l.code
-- 3. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';''')

,city,language
0,Hyderabad (India),Assamese
1,Hyderabad (India),Bengali
2,Hyderabad (India),Gujarati
3,Hyderabad (India),Hindi
4,Hyderabad (India),Kannada
5,Hyderabad (India),Maithili
6,Hyderabad (India),Malayalam
7,Hyderabad (India),Marathi
8,Hyderabad (India),Oriya
9,Hyderabad (India),Other


In [19]:
sql_query('''-- Select fields
SELECT country_name as country, region, life_expectancy AS life_exp
-- From countries (alias as c)
FROM countries AS c
  -- Join to populations (alias as p)
  LEFT JOIN populations as p
    -- Match on country code
    ON c.code = p.country_code
-- Focus on 2010
WHERE year = '2010' AND life_expectancy IS NOT NULL
-- Order by life_exp
ORDER BY life_expectancy
-- Limit to 5 records
LIMIT 5;''')

,country,region,life_exp
0,Lesotho,Southern Africa,47.4834146341463
1,Central African Republic,Central Africa,47.6253170731707
2,Sierra Leone,Western Africa,48.2289512195122
3,Swaziland,Southern Africa,48.345756097561
4,Zimbabwe,Eastern Africa,49.5746585365854
